## SP23:CS 477/577: Python for Machine Learning

### HW2: Use logistic regression for classification


Student Name: Blanchette, Daniel

### 1. Preliminary

#### 1.1 Problem formulation
1. Training set. Data sampels: X_train = {$x_1, x_2, ..., x_i, ..., x_m$}. Target (class labels):Y_train = {$y_1, y_2, ..., y_i, ..., y_m$}
    
2. Predicted class probability in logistic regression:

\begin{equation*} 
    \hat{y_i} = 1/(1+exp-(w_0 + w_1*x_{i,1} + ,..., +w_n*x_{i,n}))
\end{equation*}    
    where $w_0, w_1,...,w_n$ are the linear model parameters, and $ x_{i,1}..., x_{i,n}$ are the features of the ith data feature vector $x_i$

3. Cost/loss function of the logistic regression: the criterion to quantitatively evaluate how good the current model is (the less the better).
\begin{equation*}
    J(w) = -C [\sum_{i=1}^m (y_i \times log(\hat{y_i}) + (1-y_i) \times log(1-\hat{y_i}))] 
\end{equation*}
    - In the above equation, $\hat{y_i}$  and  $y_i$ are the predicted class lable and the true class label of data sample $x_i$, respectively.
    - P is the penalty/regularizer, and C controls the contribution of P. There are three typical options of P: L1 norm ($ \|w\|_1$), L2 norm ($\frac{1}{2}w^T w$) and elstic-Net($\frac{1 - \rho}{2}w^T w + \rho \|w\|_1$)
    
4. Model training is to use an 'optimization algorithm' to find the best $w$ that can minimize the loss function $J(w)$. Refer to the gradient descent algorithm in the Other Learning materials for more information.

#### 1.2. The sklearn.linear_model.LogisticRegression class

1. class introduction: Key attributes (e.g., C, and penalty)and methods (fit, predict, predict_proba, and score)
    - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#
    
2. source code
    - https://github.com/scikit-learn/scikit-learn/blob/b194674c4/sklearn/linear_model/_logistic.py#L1191

### 2. Breast Cancer Detection

Develop the logistic regression approach to classify breast cases into malignat(cancer) and benign.

Dataset

    :Number of data samples: 569
    :Number of features: 30 numeric. The first 10 features were directly calculated using mean feautues of all nuclei in an image
    :Class labels (We changed the original class labels by using 1 - y.)
        : Malignant: 1 (malignant or cancer)
        : Benign: 0  (benign or non-cancer)  
    https://scikit-learn.org/stable/datasets/index.html#breast-cancer-dataset
    
    

#### 2.1 Prepare dataset

In [148]:
import sklearn.datasets as ds
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# 1) data preparation
X,y = ds.load_breast_cancer(return_X_y=True)
y = 1-y
print('Dataset:', X.shape, y.shape)

rs = 0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = rs)
print('training samples', X_train.shape, 'test samples', X_test.shape)
print('Xtest', y_train)

Dataset: (569, 30) (569,)
training samples (455, 30) test samples (114, 30)
Xtest [0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 0 1 0 0 0
 0 0 0 0 1 0 1 0 1 1 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0
 1 0 0 1 1 0 0 1 1 0 0 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 1 0 1 0 0
 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 1 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0
 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 1
 1 1 1 0 1 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 1 1 0 0 1 0 0 0 1 0 1 0 1
 1 1 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 0 1 0 0 0 1 1 0 0 0 0 0
 1 1 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 1 1 0 0 0 0 0 1 1 1 0 0 1
 1 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 1 1 1 0 1 0 1 0 1
 1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 1
 1 1 1 0 1 1 1 0 1 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0 0 1 0 0 1
 0 0 0 0 0 1 1 1 0

#### 2.2 Train and explore a logistic regression model. 25 points

In [85]:
#1) train a logistic regression model. 5 points
clf = LR(penalty='none', random_state=0, max_iter=1000, verbose=2)


#2) print out the optimal model(linear) parameters after the training. 5 points
            #w0
            #w1...n
clf.fit(X_train, y_train)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           31     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.15382D+02    |proj g|=  3.77194D+04

At iterate    1    f=  3.10664D+02    |proj g|=  1.15372D+04

At iterate    2    f=  3.06501D+02    |proj g|=  1.02634D+04

At iterate    3    f=  2.80457D+02    |proj g|=  4.26347D+04

At iterate    4    f=  2.53969D+02    |proj g|=  5.13566D+04

At iterate    5    f=  2.18351D+02    |proj g|=  4.45593D+04

At iterate    6    f=  1.83743D+02    |proj g|=  2.98168D+04

At iterate    7    f=  1.58099D+02    |proj g|=  1.80991D+04

At iterate    8    f=  1.41681D+02    |proj g|=  9.94303D+03

At iterate    9    f=  1.32666D+02    |proj g|=  4.51734D+03

At iterate   10    f=  1.28764D+02    |proj g|=  1.83151D+03

At iterate   11    f=  1.27171D+02    |proj g|=  1.63943D+03

At iterate   12    f=  1.26023D+02    |proj g|=  2.97649D+03

At iterate   13    f=  1.2

/home/tarnished-dan22/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s finished


LogisticRegression(max_iter=1000, penalty='none', random_state=0, verbose=2)

.49404D+01    |proj g|=  2.71934D+01

At iterate  850    f=  3.49191D+01    |proj g|=  9.07199D+01

At iterate  851    f=  3.48951D+01    |proj g|=  8.80111D+01

At iterate  852    f=  3.48949D+01    |proj g|=  1.08273D+02

At iterate  853    f=  3.48591D+01    |proj g|=  1.00043D+02

At iterate  854    f=  3.48471D+01    |proj g|=  1.72993D+02

At iterate  855    f=  3.48004D+01    |proj g|=  8.99188D+01

At iterate  856    f=  3.47661D+01    |proj g|=  1.73221D+01

At iterate  857    f=  3.47538D+01    |proj g|=  7.60765D+01

At iterate  858    f=  3.47496D+01    |proj g|=  5.39588D+01

At iterate  859    f=  3.47487D+01    |proj g|=  6.22608D+01

At iterate  860    f=  3.47472D+01    |proj g|=  7.79559D+00

At iterate  861    f=  3.47470D+01    |proj g|=  4.91941D+00

At iterate  862    f=  3.47465D+01    |proj g|=  4.91275D+00

At iterate  863    f=  3.47457D+01    |proj g|=  2.24622D+01

At iterate  864    f=  3.47454D+01    |proj g|=  2.18502D+01

At iterate  865    f=  3.47446D+

In [91]:
#3) calculate and print out the predicted class probabilities of 
#the first 5 training samples by implementing the equation in 1.1 (y_i^hat). 10 points
y_pred = clf.predict(X_train[:5])
print(y_train[:5])

[0 0 0 0 0]


In [87]:
#4) calculate and print out the predicted class probabilities of 
#the first 5 training samples using the predict_proba() function. 5 points
clf.predict_proba(X_train[:5])

array([[9.99522241e-01, 4.77759236e-04],
       [9.54101126e-01, 4.58988741e-02],
       [9.66440302e-01, 3.35596984e-02],
       [9.99975486e-01, 2.45137792e-05],
       [9.45031926e-01, 5.49680744e-02]])

#### 2.3 Evaluation 1.0. 25 points

1) Accuracy. 5 points

In [88]:
# calculate and print out the accuracy of the trained model on the training set
# using the score function
#tst_acc = clf.score(X_test, y_test)
train_acc = clf.score(X_train, y_train)

#print('Test Accuracy:', tst_acc)
print('Train Accuracy:', train_acc)

Test Accuracy: 0.956140350877193
Train Accuracy: 0.9692307692307692


2) Confusion matrix. 5 points

In [129]:
from sklearn.metrics import confusion_matrix
#print out the confusion matrix (cfm) for the trained model on the test set

y_pred1 = clf.predict(X_test)
cfm = confusion_matrix(y_test, y_pred1)
print(cfm)
# additional print out of positions in the matrix to help me keep things straight for the ratios
#tn, fp, fn, tp =  confusion_matrix(y_test, y_pred1).ravel()
#print('\n', tn, fp, fn, tp)

[[63  4]
 [ 1 46]]


3) What is your observation from the cfm? 5 points

Response (add your response here): 
   
   a) By inspection, it appears that this model is out of balance as each row does not total to 1.00
   
   b) Due to to CFM being out of balance, it implies that the model could skew the predictions to appear to be  more accurate than they actually are.

4) Recall and Precision. 10 points

In [114]:
#print out the recall ratio of the trained model for 
#detecting breast cancer using the above cfm

# print out the precision of the trained model for 
#detecting breast cancer using the above cfm


# using classification_report to cover both print requirements and show a more report for the data.
from sklearn.metrics import classification_report
target_names = ['benign', 'malignant']
print(classification_report(y_test, y_pred, target_names = target_names))


              precision    recall  f1-score   support

      benign       0.98      0.94      0.96        67
   malignant       0.92      0.98      0.95        47

    accuracy                           0.96       114
   macro avg       0.95      0.96      0.96       114
weighted avg       0.96      0.96      0.96       114



####  2.4 Evaluation 2.0. 30 points

K-fold cross validation. (See Lecture Notes 10)

1) Why do we need K-fold cross validation in this application? 5 points

    Reponse(add your response here):
    So that we can evaluate our model by training several other models on subsets of our cancer
    data. K-fold cross validation is especially important because as in the previous parts we noticed that we
    have bias in our model which is scewing our results to appear more accurate than they are. With K-fold, 
    we can apply its methods to the training set to further train it. We would never use the test data set for
    this because it is important to preserve for comparisons later on to see how well the cross validation has
    worked.
    


2) Use the 5-fold cross validation to re-evaluate a logistic regression model. 15 points

In [171]:
#cross_validate in sklearn should be used in this task
from sklearn.model_selection import cross_val_score as cvs

myLR_1 = LogisticRegression(C = 5, penalty = 'l1', solver = 'saga', max_iter=3100, random_state = rs)

#call the cross_validate function and set scoring = ('accuracy', 'recall')
scores = cvs(myLR_1, X_train, y_train, scoring="accuracy", cv=10)

#print out the test accuracy for each fold and print out the mean accuracy 
print('Cross Validation Accuracy Scores: ', scores)
mean_scores = cvs(myLR_1, X_train, y_train, scoring="accuracy", cv=10).mean()
print('Mean Score: ', mean_scores)

#print out the test recall ratios for each fold and print out the mean recall ratio 
print('Recall Scores: ', cvs(myLR_1, X_train, y_train, scoring = "recall", cv=10))
print('Mean Recall: ', cvs(myLR_1, X_train, y_train, scoring = "recall", cv=10).mean())



Cross Validation Accuracy Scores:  [0.95652174 0.91304348 0.89130435 0.89130435 0.86956522 0.95555556
 0.95555556 0.84444444 0.95555556 0.88888889]
Mean Score:  0.9121739130434783
Recall Scores:  [0.94117647 0.82352941 0.76470588 0.76470588 0.76470588 0.9375
 0.9375     0.75       0.875      0.75      ]
Mean Recall:  0.8308823529411764


3) What are the differences between Evaluation 1.0 and Evaluation 2.0? Which one is better for this application? and why? 10 points

Response (Add your responses here):

(1) Our first evaluation gave us 90% or higher results across recall, precision, and accuracy for our model. We recognized that our confusion matrix was out of balance and was skewing the data to reflect those numbers.

(2) As we observe, since we applied cross validation the mean recall and accuracy are drastically different from our original set. Recall was in the upper 90% range but after we applied k-fold cross validation it is now roughly 91%. We also noticed the accuracy was also higher at 95-96% after cross validation, we observe that it is 83%. This implies that cross validation is doing exactly what we expected it to do, it is training our model based on data subsets and providing less bias results.




#### 2.5 How can we improve the recall ratio?  20 points.

Achieving high recall ratio(RR) is very important is cancer detection. Because RR indicates a model's ability to identify cancers; and low recall ratio may cost the lives of patients. Can you help improve the logistic regression approach to increse the recall ratio?

In [172]:
#train a logistic regression model.
myLR_2 = LR(penalty = 'l1', C = 5.0, solver = 'saga', max_iter=3000, random_state = 0)
myLR_2.fit(X_train, y_train)

LogisticRegression(C=5.0, max_iter=3000, penalty='l1', random_state=0,
                   solver='saga')

1) Print out the predicted probabilities of all misclassified samples (test set) and print out their class labels. 5 points

In [178]:
print('predicted probabilites: ', clf.predict_proba(X_test))
print('class_labels: ', clf.classes_)

predicted probabilites:  [[1.74963557e-03 9.98250364e-01]
 [9.88023707e-01 1.19762925e-02]
 [9.99865113e-01 1.34886780e-04]
 [9.41260328e-01 5.87396722e-02]
 [9.99999262e-01 7.37554953e-07]
 [9.99389277e-01 6.10722739e-04]
 [9.99444887e-01 5.55113390e-04]
 [9.99846728e-01 1.53271501e-04]
 [9.86060326e-01 1.39396744e-02]
 [9.99996837e-01 3.16317766e-06]
 [7.26818250e-01 2.73181750e-01]
 [8.62245600e-01 1.37754400e-01]
 [9.99819990e-01 1.80009766e-04]
 [2.76411366e-01 7.23588634e-01]
 [7.82333986e-01 2.17666014e-01]
 [2.72926222e-03 9.97270738e-01]
 [9.75561127e-01 2.44388727e-02]
 [5.37325739e-12 1.00000000e+00]
 [5.22888348e-05 9.99947711e-01]
 [4.44089210e-16 1.00000000e+00]
 [1.50481152e-05 9.99984952e-01]
 [1.73101803e-02 9.82689820e-01]
 [9.99374331e-01 6.25669495e-04]
 [9.98464278e-01 1.53572233e-03]
 [5.18707949e-05 9.99948129e-01]
 [9.98698786e-01 1.30121424e-03]
 [9.99903917e-01 9.60830061e-05]
 [2.71577922e-02 9.72842208e-01]
 [9.99471667e-01 5.28333486e-04]
 [3.73256981e-13 1

2) Define a new prediction function that use threshod $th=0.37$ to determine the final class label (0 0r 1). 10 points

Tips: if the predict_proba function returns a probability that is less than $th$ the new prediction function returns 0, otherwise returns 1.

In [236]:
def myPredict(lr, X, th = 0.35):
    '''  
        input:
            lr: a trained logistic regression model
            X: input feature vectors
            th: threshold. 

        return predicted binary labels for all inuput samples
    '''    
    #add your code here
    pred_b = []
    prob_score = lr.predict_proba(X)
    for i in prob_score:
        if prob_score.any() < th :
            pred_b.append(0) 
            #print('test')
        else:
            pred_b.append(1)
            #print('nay')


    return pred_b

y_pred = myPredict(myLR_2, X_test, th = 0.35)
# print(y_pred)
cm = confusion_matrix(y, y_pred)
print(cm)

#print out the accuracy and the recall ratio on the test set
print('Recall Scores: ', cvs(myLR_2, X_train, y_train, scoring = "recall", cv=10))
print('Accuracy Scores:', cvs(myLR_2, X_train, y_train, scoring="accuracy", cv=10)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


ValueError: Found input variables with inconsistent numbers of samples: [569, 114]

3) Other possible ideas could be applied to improve the recall ratio of the logistic regression. 5 points

Response:

1) Use repeated random test-train splits to validate your model.

2) Sklearn's gridsearch parameter called 'refit' allows us to specify a certain metric and maximize the outcome.